In [1]:
import pickle
import json
import csv
from dateutil.parser import parse
from datetime import timedelta
import pandas as pd
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', None)

In [2]:
"""读worker_quality并(0,1)归一化"""

worker_quality = {}
csvfile = open("../data/worker_quality.csv", "r")
csvreader = csv.reader(csvfile)
for line in csvreader:
    if line[0] != 'worker_id':  # 防止读到第一行
        if float(line[1]) > 0.0:
            worker_quality[int(line[0])] = float(line[1]) / 100.0
csvfile.close()
worker_id_list = list(worker_quality.keys())
worker_quality

{2527235: 0.73,
 2531356: 0.73,
 2400287: 0.75,
 2523170: 0.71,
 2490406: 0.74,
 2523178: 0.73,
 2531376: 0.83,
 3067913: 0.73,
 2523196: 0.73,
 3063875: 0.73,
 3080263: 0.73,
 2539595: 0.74,
 1286229: 0.82,
 2523233: 0.73,
 2383975: 0.85,
 2531432: 0.73,
 2424945: 0.77,
 3067923: 0.73,
 2375806: 0.77,
 2523267: 0.73,
 2375814: 0.85,
 2498696: 0.82,
 2498702: 0.84,
 3072150: 0.73,
 2523291: 0.91,
 2523295: 0.73,
 2474147: 0.75,
 2375847: 0.97,
 3072172: 0.71,
 2408623: 0.95,
 2416818: 0.8,
 2523317: 0.73,
 570740: 0.82,
 2490564: 0.76,
 1482954: 0.91,
 3072209: 0.76,
 2539730: 0.94,
 3078862: 0.55,
 2433238: 0.85,
 3072219: 0.78,
 1593330: 0.78,
 2457826: 0.82,
 2506980: 0.81,
 2538193: 0.73,
 3064041: 0.73,
 3080428: 0.72,
 1777901: 0.82,
 2302198: 0.88,
 2343159: 1.0,
 2507001: 0.74,
 2515204: 0.95,
 3072266: 0.95,
 2359578: 0.82,
 3080482: 0.73,
 2418054: 0.87,
 3072300: 0.89,
 2515258: 0.73,
 2351420: 0.91,
 2457935: 0.88,
 2515282: 0.86,
 2425178: 0.88,
 2425183: 0.89,
 2539872: 0

In [3]:
"""读project和entry信息"""

file = open("../data/project_list.csv", "r")
project_list_lines = file.readlines()
file.close()
project_dir = "../data/project/"
entry_dir = "../data/entry/"

# all_begin_time = parse("2018-01-01T0:0:0Z")
project_info = {}
entry_info = {}
limit = 24
category_list = {}
industry_list = {}
tmp_category_list = []
tmp_subcategory_list = []
for line in project_list_lines:
    line = line.strip('\n').split(',')
    project_id = int(line[0])  # 项目id
    entry_count = int(line[1])  # 参与人次

    with open(project_dir + "project_" + str(project_id) + ".txt", 'r', encoding='utf-8') as file:
        htmlcode = file.read()
    text = json.loads(htmlcode)
    if text["category"] not in tmp_category_list:
        tmp_category_list.append(text["category"])
    if text["sub_category"] not in tmp_subcategory_list:
        tmp_subcategory_list.append(text["sub_category"])

    project_info[project_id] = {}
    project_info[project_id]["start_date"] = parse(text["start_date"])  # project start date
    project_info[project_id]["deadline"] = parse(text["deadline"])  # project end date
    if text["category"] not in category_list:
        category_list[text["category"]] = len(category_list)
    project_info[project_id]["category"] = category_list[text["category"]]  # project category
#     project_info[project_id]["sub_category"] = int(text["sub_category"])  # project sub_category
    project_info[project_id]["award"] = float(text["total_awards"])
    if text["industry"] not in industry_list:
        industry_list[text["industry"]] = len(industry_list)
    project_info[project_id]["industry"] = industry_list[text["industry"]]  # project domain
    project_info[project_id]["entry_count"] = int(text["entry_count"])  # project answers in total

    entry_info[project_id] = {}
    k = 0
    print("project_id:", project_id)
    while k < entry_count:
        with open(entry_dir + "entry_" + str(project_id) + "_" + str(k) + ".txt", 'r', encoding='utf-8') as file:
            htmlcode = file.read()
        text = json.loads(htmlcode)

        for item in text["results"]:
            entry_number = int(item["entry_number"])
            entry_info[project_id][entry_number] = {}
            entry_info[project_id][entry_number]["entry_created_at"] = parse(item["entry_created_at"])  # worker answer_time
            entry_info[project_id][entry_number]["worker"] = int(item["author"])  # work_id
        k += limit
        
project_id_list = list(entry_info.keys())
print(len(project_id_list))

project_id: 2908089
project_id: 2909696
project_id: 2909401
project_id: 2909378
project_id: 2909457
project_id: 2909391
project_id: 2909411
project_id: 2898285
project_id: 2892334
project_id: 2909355
project_id: 2908220
project_id: 2908835
project_id: 2908405
project_id: 2909574
project_id: 2906513
project_id: 2909282
project_id: 2908900
project_id: 2909375
project_id: 2909222
project_id: 2909354
project_id: 2909367
project_id: 2909528
project_id: 2907331
project_id: 2909312
project_id: 2909288
project_id: 2909330
project_id: 2909177
project_id: 2909105
project_id: 2909568
project_id: 2908278
project_id: 2906345
project_id: 2907820
project_id: 2908916
project_id: 2908982
project_id: 2907917
project_id: 2907700
project_id: 2909181
project_id: 2909291
project_id: 2908987
project_id: 2908793
project_id: 2902933
project_id: 2909007
project_id: 2907134
project_id: 2909310
project_id: 2908854
project_id: 2909246
project_id: 2908862
project_id: 2909245
project_id: 2909515
project_id: 2906265


In [4]:
"""将所有completed tasks的信息保存为list"""

complete_info_list = []  # 保存每个worker 每参与一个project 就有一次entry的记录
for w_id in worker_id_list:
    entry_cnt = 0
    for p_id in project_id_list:
        for e_id, e_info in entry_info[p_id].items():
            if e_info['worker'] == w_id:
                entry_cnt += 1
                complete_info = {}
                complete_info['entry_time'] = e_info['entry_created_at']
                complete_info['worker_id'] = w_id
                complete_info['worker_quality'] = worker_quality[w_id]
                complete_info['project_id'] = p_id
                complete_info['start_time'] = project_info[p_id]['start_date']
                complete_info['end_time'] = project_info[p_id]['deadline']
                complete_info['category'] = project_info[p_id]['category']
                complete_info['industry'] = project_info[p_id]['industry']
                complete_info['award'] = project_info[p_id]['award']
                complete_info['complete'] = 1  # 代表worker完成了该project
                complete_info_list.append(complete_info)
                break  # 对于每个worker 在同一个project中 只读取最晚提交的一次记录
    print('worker_id:', w_id, 'entry_num:', entry_cnt)  # 每个worker参与project(entry)的次数

"""修正少部分entry_time晚于end_time的记录"""
for i in range(len(complete_info_list)):
    if complete_info_list[i]['entry_time'] > complete_info_list[i]['end_time']:
        complete_info_list[i]['entry_time'] = complete_info_list[i]['entry_time'] - timedelta(minutes=1)  # 减一分钟
    if complete_info_list[i]['entry_time'] > complete_info_list[i]['end_time']:
        print(complete_info_list[i])

worker_id: 2527235 entry_num: 3
worker_id: 2531356 entry_num: 6
worker_id: 2400287 entry_num: 1
worker_id: 2523170 entry_num: 1
worker_id: 2490406 entry_num: 5
worker_id: 2523178 entry_num: 8
worker_id: 2531376 entry_num: 86
worker_id: 3067913 entry_num: 1
worker_id: 2523196 entry_num: 3
worker_id: 3063875 entry_num: 1
worker_id: 3080263 entry_num: 3
worker_id: 2539595 entry_num: 1
worker_id: 1286229 entry_num: 60
worker_id: 2523233 entry_num: 3
worker_id: 2383975 entry_num: 9
worker_id: 2531432 entry_num: 1
worker_id: 2424945 entry_num: 8
worker_id: 3067923 entry_num: 1
worker_id: 2375806 entry_num: 1
worker_id: 2523267 entry_num: 1
worker_id: 2375814 entry_num: 13
worker_id: 2498696 entry_num: 56
worker_id: 2498702 entry_num: 5
worker_id: 3072150 entry_num: 7
worker_id: 2523291 entry_num: 63
worker_id: 2523295 entry_num: 7
worker_id: 2474147 entry_num: 8
worker_id: 2375847 entry_num: 86
worker_id: 3072172 entry_num: 2
worker_id: 2408623 entry_num: 84
worker_id: 2416818 entry_num: 9
w

In [5]:
"""按entry_time升序排列"""

complete_info_df = pd.DataFrame(complete_info_list)
complete_info_list = sorted(complete_info_list, key=lambda x: x['entry_time'])
tmp_df = complete_info_df.sort_values(by=['entry_time'], ascending=[True]).copy()
tmp_df

,entry_time,worker_id,worker_quality,project_id,start_time,end_time,category,industry,award,complete
33813,2017-06-08 11:24:39+00:00,2536361,0.91,2407596,2017-06-08 08:35:59+00:00,2018-04-24 12:23:28.457022+00:00,0,8,200.0,1
45162,2017-06-08 13:22:36+00:00,2300135,0.97,2407596,2017-06-08 08:35:59+00:00,2018-04-24 12:23:28.457022+00:00,0,8,200.0,1
54980,2017-06-08 13:45:42+00:00,1531451,0.90,2407596,2017-06-08 08:35:59+00:00,2018-04-24 12:23:28.457022+00:00,0,8,200.0,1
1620,2017-06-08 14:59:34+00:00,2539730,0.94,2407596,2017-06-08 08:35:59+00:00,2018-04-24 12:23:28.457022+00:00,0,8,200.0,1
40954,2017-06-08 17:24:16+00:00,2447058,0.96,2407596,2017-06-08 08:35:59+00:00,2018-04-24 12:23:28.457022+00:00,0,8,200.0,1
...,...,...,...,...,...,...,...,...,...,...
22163,2019-03-03 14:45:06.478370+00:00,2313223,0.84,2909696,2019-02-28 19:40:45.627395+00:00,2019-03-03 19:40:45.627395+00:00,0,1,200.0,1
43367,2019-03-03 15:34:24.981871+00:00,2521125,0.99,2909696,2019-02-28 19:40:45.627395+00:00,2019-03-03 19:40:45.627395+00:00,0,1,200.0,1
14990,2019-03-03 17:08:21.534953+00:00,3057696,0.96,2909696,2019-02-28 19:40:45.627395+00:00,2019-03-03 19:40:45.627395+00:00,0,1,200.0,1
55585,2019-03-03 17:17:54.763527+00:00,2522799,0.89,2909696,2019-02-28 19:40:45.627395+00:00,2019-03-03 19:40:45.627395+00:00,0,1,200.0,1


In [6]:
"""依据project_id查找参与该project的worker_id和entry_time"""

project_worker_dict = {}
for project_id in project_id_list:
    worker_entry_info_list = []
    for i in range(len(complete_info_list)):
        if complete_info_list[i]['project_id'] == project_id:
            worker_entry_info_list.append({'worker_id': complete_info_list[i]['worker_id'],
                                           'entry_time': complete_info_list[i]['entry_time']})
    project_worker_dict[project_id] = worker_entry_info_list

### MDP(w)
【S】对于worker i，i时刻所有可参与的project category,industry,award + 近n次完成的project的平均category,industry,award + worker_quality + project_quality  
【A】p_i  
【R】complete p_i=1 else 0  
【S'】同样是worker i，i+1时刻所有可参与的project category,industry,award + 近n次完成的project的平均category,industry,award + worker_quality + project_quality  

### MDP(r)
【S】同MDP(w)S  
【A】同MDP(w)A  
【R】complere p_=quality_gain else 0  
【S'】下一时刻任意worker的entry对应的S  

In [18]:
'''生成MDP(w)和MDP(r)'''
    
ood_w, ood_r = [], []
test_w, test_r = [], []
FORMER_E_CNT_THRESHOLD = 5
LATTER_E_CNT_THRESHOLD = 2
DIXIT_STIGLITZ_P = 2

max_available_p = 0

cannot_train_cnt, can_train_cnt, test_cnt, ood_cnt = 0, 0, 0, 0

for i in range(len(complete_info_list)-1):
    if i % 500 == 0:
        print("处理第%d条entry记录中，共%d条记录" % (i, len(complete_info_list)-1))
    w_id = complete_info_list[i]['worker_id']
    p_id = complete_info_list[i]['project_id']
    entry_time = complete_info_list[i]['entry_time']
    
    '''校验该worker在当前entry之前是否有n次记录 & 下一moment的worker在下一moment的entry之前是否有n次记录'''
    former_e_info_list = []  # 保存当前worker之前的entry
    moment_former_e_info_list = []  # 保存下一moment的worker之前的entry
    for j in range(i):
        if w_id == complete_info_list[j]['worker_id']:  # 当前第j条entry是否是当前时刻的worker
            e_time = complete_info_list[j]['entry_time']
            former_e_info_list.append({'project_id': complete_info_list[j]['project_id'],
                                       'category': complete_info_list[j]['category'],
                                       'industry': complete_info_list[j]['industry'],
                                       'award': complete_info_list[j]['award'],
                                       'start_gap': (e_time - complete_info_list[j]['start_time']).days,
                                       'end_gap': (complete_info_list[j]['end_time'] - e_time).days})
        if complete_info_list[i+1]['worker_id'] == complete_info_list[j]['worker_id']:  # 当前第j条entry是否是下一moment的worker
            e_time = complete_info_list[j]['entry_time']
            moment_former_e_info_list.append({'project_id': complete_info_list[j]['project_id'],
                                              'category': complete_info_list[j]['category'],
                                              'industry': complete_info_list[j]['industry'],
                                              'award': complete_info_list[j]['award'],
                                              'start_gap': (e_time - complete_info_list[j]['start_time']).days,
                                              'end_gap': (complete_info_list[j]['end_time'] - e_time).days})

    if len(former_e_info_list) >= FORMER_E_CNT_THRESHOLD and \
       len(moment_former_e_info_list) >= FORMER_E_CNT_THRESHOLD:  # 有最近的n次记录才能用于训练 或用于测试
        can_train_cnt += 1
        '''校验该worker在当前entry之后的记录是否<m次'''
        latter_p_info_list = []  # 保存当前worker之后的entry
        for j in range(i+1, len(complete_info_list)-1):
            if w_id == complete_info_list[j]['worker_id']:
                e_time = complete_info_list[j]['entry_time']
                latter_p_info_list.append({'project_id': complete_info_list[j]['project_id'],
                                           'category': complete_info_list[j]['category'],
                                           'industry': complete_info_list[j]['industry'],
                                           'award': complete_info_list[j]['award'],
                                           'start_gap': (e_time - complete_info_list[j]['start_time']).days,
                                           'end_gap': (complete_info_list[j]['end_time'] - e_time).days,
                                           'entry_time': e_time})
                # if len(latter_p_info_list) >= LATTER_E_CNT_THRESHOLD:
                #     break

        # if len(latter_p_info_list) >= LATTER_E_CNT_THRESHOLD:  # 之后的记录>=m次，当前transition做训练集
        #     pass                     
        if (len(former_e_info_list) <= FORMER_E_CNT_THRESHOLD+1 or \
            len(moment_former_e_info_list) <= FORMER_E_CNT_THRESHOLD+1) and \
           len(latter_p_info_list) < LATTER_E_CNT_THRESHOLD:  # 之后的记录<m次，当前transition做测试集
            # test_cnt += 1
            ood_cnt += 1
            print("OOD!")
            '''统计当前时刻S可参与的project'''
            curr_time = entry_time
            curr_available_p_info_list = []
            for j in range(len(complete_info_list)-1):
                tmp_p_info = {'project_id':  complete_info_list[j]['project_id'],
                              'category': complete_info_list[j]['category'],
                              'industry': complete_info_list[j]['industry'],
                              'award': complete_info_list[j]['award'],
                              'start_time': complete_info_list[j]['start_time'],
                              'end_time': complete_info_list[j]['end_time'],
                              'start_gap': (curr_time - complete_info_list[j]['start_time']).days,
                              'end_gap': (complete_info_list[j]['end_time'] - curr_time).days}
                if complete_info_list[j]['start_time'] <= curr_time and curr_time <= complete_info_list[j]['end_time'] and \
                    tmp_p_info not in curr_available_p_info_list:
                    curr_available_p_info_list.append(tmp_p_info)

            '''统计每一时刻最大的project数量'''
            if len(curr_available_p_info_list) > max_available_p:
                max_available_p = len(curr_available_p_info_list)
            
            '''构建测试集的S,A,R'''
            single_state = []  # row-wise state
            action = []
            reward_w = []
            reward_r = []
            for j in range(len(curr_available_p_info_list)):
                # 【S】available
                row_state = {'a_category': curr_available_p_info_list[j]['category'],
                             'a_industry': curr_available_p_info_list[j]['industry'],
                             'a_award': curr_available_p_info_list[j]['award'],
                             'a_start_gap': curr_available_p_info_list[j]['start_gap'],
                             'a_end_gap': curr_available_p_info_list[j]['end_gap']}
                # 【S】last n
                tmp = 1
                for former_e_info in former_e_info_list[len(former_e_info_list)-FORMER_E_CNT_THRESHOLD:]:  # 取近n次entry记录
                    row_state['l'+str(tmp)+'_category'] = former_e_info['category']
                    row_state['l'+str(tmp)+'_industry'] = former_e_info['industry']
                    row_state['l'+str(tmp)+'_award'] = former_e_info['award']
                    row_state['l'+str(tmp)+'_start_gap'] = former_e_info['start_gap']
                    row_state['l'+str(tmp)+'_end_gap'] = former_e_info['end_gap']
                    tmp += 1
                # 【S】worker_quality
                row_state['worker_quality'] = worker_quality[w_id]
                # 【S】project_quality
                sum_q_pow = 0.0
                for w_e_info in project_worker_dict[p_id]:  # 取出参与当前project的所有worker
                    if w_e_info['entry_time'] < curr_time:  # 只考虑比当前worker参与得更早的worker
                        sum_q_pow += pow(worker_quality[w_e_info['worker_id']], DIXIT_STIGLITZ_P)
                row_state['project_quality'] = pow(sum_q_pow, 1.0/DIXIT_STIGLITZ_P)
                single_state.append(row_state)
                # 【A】
                action.append(j)
                # 【R】
                if p_id == curr_available_p_info_list[j]['project_id']:
                    reward_w.append(1.0)
                    old_pq = row_state['project_quality']  # 该project不包含该worker时的project_quality
                    new_pq = pow(sum_q_pow + pow(worker_quality[w_id], DIXIT_STIGLITZ_P), 
                                 1.0/DIXIT_STIGLITZ_P)  # 该project加入该worker后的project_quality
                    reward_r.append(new_pq - old_pq)  # quality_gain
                else:
                    reward_w.append(0.0)
                    reward_r.append(0.0)
            '''只保存R非0的(S,A,R)元组'''
            for j in range(len(reward_w)):
                if reward_w[j] != 0.0:
                    transition = {}
                    transition['state'] = single_state
                    transition['action'] = action[j]
                    transition['reward'] = reward_w[j]
                    # test_w.append(transition)
                    ood_w.append(transition)
            for j in range(len(reward_r)):
                if reward_r[j] != 0.0:
                    transition = {}
                    transition['state'] = single_state
                    transition['action'] = action[j]
                    transition['reward'] = reward_r[j]
                    # test_r.append(transition)
                    ood_r.append(transition)
            # print("构建测试数据……")
            # print('此刻的w,p,e', w_id, p_id, entry_time)
            # print('此刻w的之前项目信息', former_e_info_list)
            # print('此刻可选的项目', len(curr_available_p_info_list), '个')
            # print('此刻w的之后项目信息', latter_p_info_list)
            # print('STATE')
            # for idx, item in enumerate(single_state):
            #     print(idx, item)
            # print('ACTION', action)
            # print('REWARD_W', reward_w)
            # print('REWARD_R', reward_r)
            # print('test_w')
            # for key, value in test_w[0].items():
            #     print(key,value)
            # print('test_r')
            # for key, value in test_r[0].items():
            #     print(key,value)
            # 1/0
    else:  # 没有最近的n次记录，无法训练
        cannot_train_cnt += 1
        continue

print('最多动作', max_available_p)
print('无法训练', cannot_train_cnt)
print('可以训练', can_train_cnt)
print('OOD', ood_cnt)

处理第0条entry记录中，共58903条记录
处理第500条entry记录中，共58903条记录
处理第1000条entry记录中，共58903条记录
处理第1500条entry记录中，共58903条记录
处理第2000条entry记录中，共58903条记录
处理第2500条entry记录中，共58903条记录
处理第3000条entry记录中，共58903条记录
OOD!
处理第3500条entry记录中，共58903条记录
处理第4000条entry记录中，共58903条记录
OOD!
OOD!
处理第4500条entry记录中，共58903条记录
OOD!
OOD!
OOD!
处理第5000条entry记录中，共58903条记录
OOD!
处理第5500条entry记录中，共58903条记录
OOD!
OOD!
OOD!
处理第6000条entry记录中，共58903条记录
OOD!
OOD!
处理第6500条entry记录中，共58903条记录
OOD!
OOD!
处理第7000条entry记录中，共58903条记录
OOD!
OOD!
处理第7500条entry记录中，共58903条记录
OOD!
处理第8000条entry记录中，共58903条记录
OOD!
处理第8500条entry记录中，共58903条记录
处理第9000条entry记录中，共58903条记录
处理第9500条entry记录中，共58903条记录
OOD!
处理第10000条entry记录中，共58903条记录
处理第10500条entry记录中，共58903条记录
OOD!
处理第11000条entry记录中，共58903条记录
OOD!
处理第11500条entry记录中，共58903条记录
处理第12000条entry记录中，共58903条记录
处理第12500条entry记录中，共58903条记录
OOD!
OOD!
处理第13000条entry记录中，共58903条记录
处理第13500条entry记录中，共58903条记录
OOD!
OOD!
处理第14000条entry记录中，共58903条记录
处理第14500条entry记录中，共58903条记录
处理第15000条entry记录中，共58903条记录
处理第15500条entry记录中，共58903条记录
OOD

In [19]:
len(ood_w)

231

In [20]:
len(ood_r)

231

In [21]:
ood_w[0]

{'state': [{'a_category': 0,
   'a_industry': 8,
   'a_award': 200.0,
   'a_start_gap': 227,
   'a_end_gap': 92,
   'l1_category': 0,
   'l1_industry': 31,
   'l1_award': 200.0,
   'l1_start_gap': 5,
   'l1_end_gap': 1,
   'l2_category': 0,
   'l2_industry': 3,
   'l2_award': 250.0,
   'l2_start_gap': 3,
   'l2_end_gap': 0,
   'l3_category': 0,
   'l3_industry': 3,
   'l3_award': 500.0,
   'l3_start_gap': 14,
   'l3_end_gap': 2,
   'l4_category': 0,
   'l4_industry': 7,
   'l4_award': 300.0,
   'l4_start_gap': 19,
   'l4_end_gap': 0,
   'l5_category': 0,
   'l5_industry': 2,
   'l5_award': 200.0,
   'l5_start_gap': 2,
   'l5_end_gap': 4,
   'worker_quality': 0.73,
   'project_quality': 3.373484845082307},
  {'a_category': 0,
   'a_industry': 7,
   'a_award': 200.0,
   'a_start_gap': 205,
   'a_end_gap': 100,
   'l1_category': 0,
   'l1_industry': 31,
   'l1_award': 200.0,
   'l1_start_gap': 5,
   'l1_end_gap': 1,
   'l2_category': 0,
   'l2_industry': 3,
   'l2_award': 250.0,
   'l2_st

In [22]:
ood_r[0]

{'state': [{'a_category': 0,
   'a_industry': 8,
   'a_award': 200.0,
   'a_start_gap': 227,
   'a_end_gap': 92,
   'l1_category': 0,
   'l1_industry': 31,
   'l1_award': 200.0,
   'l1_start_gap': 5,
   'l1_end_gap': 1,
   'l2_category': 0,
   'l2_industry': 3,
   'l2_award': 250.0,
   'l2_start_gap': 3,
   'l2_end_gap': 0,
   'l3_category': 0,
   'l3_industry': 3,
   'l3_award': 500.0,
   'l3_start_gap': 14,
   'l3_end_gap': 2,
   'l4_category': 0,
   'l4_industry': 7,
   'l4_award': 300.0,
   'l4_start_gap': 19,
   'l4_end_gap': 0,
   'l5_category': 0,
   'l5_industry': 2,
   'l5_award': 200.0,
   'l5_start_gap': 2,
   'l5_end_gap': 4,
   'worker_quality': 0.73,
   'project_quality': 3.373484845082307},
  {'a_category': 0,
   'a_industry': 7,
   'a_award': 200.0,
   'a_start_gap': 205,
   'a_end_gap': 100,
   'l1_category': 0,
   'l1_industry': 31,
   'l1_award': 200.0,
   'l1_start_gap': 5,
   'l1_end_gap': 1,
   'l2_category': 0,
   'l2_industry': 3,
   'l2_award': 250.0,
   'l2_st

In [17]:
'''将ood写入pkl文件'''
ood_filename = "../data/ood_"+str(len(ood_w))+".pkl"
with open(ood_filename, "wb") as f:
    pickle.dump({'ood_w': ood_w, 'ood_r': ood_r}, f)
